In [ ]:
import pandas as pd
import numpy as np
df=pd.read_csv('final_players_ipl_2020.csv')
df

In [10]:

df2 = pd.read_csv('batting.csv')  # Replace with the actual file name
df3 = pd.read_csv('bowling.csv')  # Replace with the actual file name

df_names = df['Player Name'].tolist()
df2_names = df2['batsmanName'].tolist()
df3_names = df3['bowlerName'].tolist()

# Check names in df against df2 and df3
df['In_df2'] = df['Player Name'].isin(df2_names)  # Adds a boolean column for df2
df['In_df3'] = df['Player Name'].isin(df3_names)  # Adds a boolean column for df3

# Alternatively, to get matching names
matching_in_df2 = df[df['In_df2']]
matching_in_df3 = df[df['In_df3']]

# Print the DataFrame with new columns indicating presence in df2 and df3
print("Original DataFrame with match indicators:")
print(df)

# Print the matching players
print("\nMatching players in df2:")
print(matching_in_df2)

print("\nMatching players in df3:")
print(matching_in_df3)

# Optionally, you can save the updated DataFrame to a new CSV
df

Original DataFrame with match indicators:
          Player Name  In_df2  In_df3
0   Bhuvneshwar Kumar    True    True
1        Chris Morris    True    True
2      Jasprit Bumrah    True    True
3         Pat Cummins    True    True
4         Trent Boult    True    True
5       Vijay Shankar    True    True
6          Ben Stokes    True    True
7      Kieron Pollard    True    True
8         Chris Gayle    True    True
9        Ishan Kishan    True   False
10     Mayank Agarwal    True   False
11  Wriddhiman Saha †    True   False

Matching players in df2:
          Player Name  In_df2  In_df3
0   Bhuvneshwar Kumar    True    True
1        Chris Morris    True    True
2      Jasprit Bumrah    True    True
3         Pat Cummins    True    True
4         Trent Boult    True    True
5       Vijay Shankar    True    True
6          Ben Stokes    True    True
7      Kieron Pollard    True    True
8         Chris Gayle    True    True
9        Ishan Kishan    True   False
10     Mayank Agarwa

,Player Name,In_df2,In_df3
0,Bhuvneshwar Kumar,True,True
1,Chris Morris,True,True
2,Jasprit Bumrah,True,True
3,Pat Cummins,True,True
4,Trent Boult,True,True
5,Vijay Shankar,True,True
6,Ben Stokes,True,True
7,Kieron Pollard,True,True
8,Chris Gayle,True,True
9,Ishan Kishan,True,False


In [11]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# Load the batting data
batting_df = pd.read_csv('batting.csv')

# Create a mapping of player names to their profile links
player_links = {}
for index, row in batting_df.iterrows():
    player_name = row['batsmanName']  # Adjust if the column name is different
    player_link = row['player_link']  # Adjust if the column name is different
    player_links[player_name] = player_link

# Function to scrape the country and playing role information from player profile
def get_player_info(profile_url):
    try:
        response = requests.get(profile_url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Fetch country information
            country_elem = soup.find('a', title=lambda x: x and "cricket team profile" in x)
            country = country_elem.find('span', class_='ds-text-title-s').text.strip() if country_elem else "Country not found"
            
            # Fetch playing role information using the provided method
            player_details = soup.find('div', class_='ds-grid lg:ds-grid-cols-3 ds-grid-cols-2 ds-gap-4 ds-mb-8')
            playing_role_elem = player_details.find('p', text='Playing Role') if player_details else None
            playing_role = playing_role_elem.find_next('span').text.strip() if playing_role_elem else "Role not found"
            
            return country, playing_role
        else:
            print(f"Failed to retrieve data for URL: {profile_url}")
    except Exception as e:
        print(f"Error fetching information for {profile_url}: {e}")
    return "Country not found", "Role not found"

# Check country and role for each player in df
countries = []
roles = []
for index, row in df.iterrows():
    player_name = row['Player Name']
    if player_name in player_links:
        profile_url = player_links[player_name]
        country, role = get_player_info(profile_url)
        countries.append(country)
        roles.append(role)
        time.sleep(1)  # Add delay to avoid overwhelming the server
    else:
        countries.append("Not Found")
        roles.append("Not Found")

# Add the country and role information to the DataFrame
df['Country'] = countries
df['Role'] = roles

# Print the updated DataFrame
print("Updated DataFrame with country and role information:")
print(df)


C:\Users\kiran\AppData\Local\Temp\ipykernel_16720\1742021238.py:29: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  playing_role_elem = player_details.find('p', text='Playing Role') if player_details else None


Updated DataFrame with country and role information:
          Player Name  In_df2  In_df3       Country                 Role
0   Bhuvneshwar Kumar    True    True         India               Bowler
1        Chris Morris    True    True  South Africa   Bowling Allrounder
2      Jasprit Bumrah    True    True         India               Bowler
3         Pat Cummins    True    True     Australia               Bowler
4         Trent Boult    True    True   New Zealand               Bowler
5       Vijay Shankar    True    True         India           Allrounder
6          Ben Stokes    True    True       England           Allrounder
7      Kieron Pollard    True    True   West Indies   Batting Allrounder
8         Chris Gayle    True    True   West Indies           Allrounder
9        Ishan Kishan    True   False         India  Wicketkeeper Batter
10     Mayank Agarwal    True   False         India               Batter
11  Wriddhiman Saha †    True   False         India  Wicketkeeper Batte

In [12]:
df = df.drop(columns=['In_df2', 'In_df3'])
df.to_csv('final_players_ipl_2020_with_info.csv', index=False)
df

,Player Name,Country,Role
0,Bhuvneshwar Kumar,India,Bowler
1,Chris Morris,South Africa,Bowling Allrounder
2,Jasprit Bumrah,India,Bowler
3,Pat Cummins,Australia,Bowler
4,Trent Boult,New Zealand,Bowler
5,Vijay Shankar,India,Allrounder
6,Ben Stokes,England,Allrounder
7,Kieron Pollard,West Indies,Batting Allrounder
8,Chris Gayle,West Indies,Allrounder
9,Ishan Kishan,India,Wicketkeeper Batter


In [13]:
import pandas as pd


# Select players with country name 'India'
indian_players = df[df['Country'] == 'India']

# Print the result
print("Indian Players:")
print(indian_players)

# Optionally, save the Indian players to a new CSV file
indian_players.to_csv('indian_players.csv', index=False)
print("Indian players saved to indian_players.csv")


Indian Players:
          Player Name Country                 Role
0   Bhuvneshwar Kumar   India               Bowler
2      Jasprit Bumrah   India               Bowler
5       Vijay Shankar   India           Allrounder
9        Ishan Kishan   India  Wicketkeeper Batter
10     Mayank Agarwal   India               Batter
11  Wriddhiman Saha †   India  Wicketkeeper Batter
Indian players saved to indian_players.csv


In [12]:
df = pd.read_csv('final_players_ipl_2020_with_info.csv')
df1 = pd.read_csv('final_batting_stats.csv')
df2 = pd.read_csv('final_bowling_stats.csv')
df_batting = df.merge(df1, left_on='Player Name', right_on='batsmanName', how='left')

# Merge the resulting dataframe with df2 (bowling stats) using Player Name and bowlerName
df_combined = df_batting.merge(df2, left_on='Player Name', right_on='bowlerName', how='left')

# Save the combined dataframe to a CSV file
df_combined.to_csv('combined_player_data.csv', index=False)

# Display the combined dataframe
print(df_combined)

          Player Name       Country                 Role        batsmanName  \
0   Bhuvneshwar Kumar         India               Bowler  Bhuvneshwar Kumar   
1        Chris Morris  South Africa   Bowling Allrounder       Chris Morris   
2      Jasprit Bumrah         India               Bowler     Jasprit Bumrah   
3         Pat Cummins     Australia               Bowler        Pat Cummins   
4         Trent Boult   New Zealand               Bowler        Trent Boult   
5       Vijay Shankar         India           Allrounder      Vijay Shankar   
6          Ben Stokes       England           Allrounder         Ben Stokes   
7      Kieron Pollard   West Indies   Batting Allrounder     Kieron Pollard   
8         Chris Gayle   West Indies           Allrounder        Chris Gayle   
9        Ishan Kishan         India  Wicketkeeper Batter       Ishan Kishan   
10     Mayank Agarwal         India               Batter     Mayank Agarwal   
11    Wriddhiman Saha         India  Wicketkeeper Ba

c:\Users\kiran\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)
